In [ ]:
pip install kneed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, auc
from sklearn.cluster import KMeans
import math
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import warnings
from kneed import KneeLocator


# Filter out the specific warning by category and message
warnings.filterwarnings("ignore", message="The default value of `n_init` will change from 10 to 'auto' in 1.4.*")


leaf_nodes=[]
child_tree = []

def binary_tree(result,final_cluster, count,tree_dic):
    first_cluster = []
    second_cluster = []
    if count==0:
        tree_dic[count]=result
    else:
        key=max(tree_dic)+1
        tree_dic[key] = result
    kmeans = KMeans(n_clusters=2, random_state=0).fit(result)

    score=0.1

    if score < 0.9 or count==0:
        for itr in range(len(result)):
            if final_cluster[itr] == 1:
                first_cluster.append(result[itr])
            else:
                second_cluster.append(result[itr])


        count = count + 1

        if len(first_cluster)> minimum_cluster_threshold :
           kmeans = KMeans(n_clusters=2, random_state=0).fit(first_cluster)



           binary_tree(first_cluster,kmeans.labels_,count,tree_dic)
        else:
            key = max(tree_dic) + 1
            tree_dic[key]=first_cluster
            leaf_nodes.append(key)

            if count <= leaf_level_threshold:

                child_tree.extend(first_cluster)


        if len(second_cluster) >minimum_cluster_threshold:
            kmeans = KMeans(n_clusters=2, random_state=0).fit(second_cluster)


            binary_tree(second_cluster,kmeans.labels_,count,tree_dic)
        else:
            key = max(tree_dic) + 1
            tree_dic[key] = second_cluster
            leaf_nodes.append(key)

            if count <= leaf_level_threshold:
                child_tree.extend(second_cluster)


    else:
        key = max(tree_dic)
        leaf_nodes.append(key)



def get_large_cluster_centroid(small_cluster_threshold,tree_dic):
    merged_clusters = {}
    mergeditr = 0
    for key in tree_dic:
        if  key in leaf_nodes :
            if len(tree_dic[key]) > small_cluster_threshold:
               merged_clusters[mergeditr] = tree_dic[key]
               mergeditr = mergeditr + 1
    final_array = []
    count = 0
    for key in merged_clusters:
        for itr in range(len(merged_clusters[key])):
            array = merged_clusters[key]
            final_array.insert(count, array[itr])
            count = count + 1
    '''merged_clustersarray=util.convert_dict_to_list(merged_clusters,len(merged_clusters),len(tree_dic[0]))'''
    centroid = np.mean(final_array, axis=0)
    return centroid

def get_large_cluster_center(small_cluster_threshold,tree_dic):
    merged_clusters = {}
    centroid_array=[]
    mergeditr = 0
    for key in tree_dic:
        if key in leaf_nodes:
            if len(tree_dic[key]) > small_cluster_threshold:
                merged_clusters[mergeditr] = tree_dic[key]
                mergeditr = mergeditr + 1

    count = 0
    for key in merged_clusters:
        centroid = np.mean(merged_clusters[key], axis=0)
        centroid_array.insert(count,centroid)
        count = count + 1
    '''this is returning the cluster centers of all the large clusters'''

    return centroid_array

def get_large_clusters_anomalyscore(tree_dic,small_cluster_threshold):
    large_clusters={}
    anomaly_largeclusters={}
    for key in tree_dic:
        if key in leaf_nodes:
           if len(tree_dic[key])>=small_cluster_threshold:
               large_clusters[key]=tree_dic[key]

    for key in large_clusters:
        array = large_clusters[key]
        centroid = np.mean(array, axis=0)
        for itr in range(len(large_clusters[key])):
            anomalyscore_largeclusters=np.linalg.norm(array[itr]-centroid)
            anomaly_largeclusters[anomalyscore_largeclusters]=array[itr]

    return anomaly_largeclusters

if __name__ == '__main__':
    ts = pd.read_csv(filename)
    ts=np.array(ts)
    st_ts = ts.tolist()
    st_ts = sorted(st_ts)
    st_ts = np.array(st_ts)
    actual = st_ts[:, -1]

    ts = pd.read_csv(filename, usecols=lambda column: column != 'Class')
    ts=np.array(ts)
    print(ts.shape)
    count = 0
    tree_dic = {}
    data_size=len(ts)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(ts)
    small_cluster_threshold = math.floor(.02 * len(ts))
    leaf_level_threshold=3
    minimum_cluster_threshold = math.floor(.1 * len(ts))

    final_cluster = kmeans.labels_
    binary_tree(ts, final_cluster, count, tree_dic)
    minkey = math.inf
    total_cluster_length = 0
    #total cluster length of the leaf clusters together,cal min key in the leaf cluster
    for key in tree_dic:
        if key in leaf_nodes:
            total_cluster_length = total_cluster_length + len(tree_dic[key])
            if key < minkey:
               minkey=key

    #identifying small clusters in the data and merging them into a single list (final_array). It then calculates the centroid of these merged data points,
    anomalyscore_dict={}
    anomalyscore_from_largeclusters={}
    merged_clusters={}
    mergeditr=0
    for key in tree_dic:
        if len(tree_dic[key])<small_cluster_threshold:
            merged_clusters[mergeditr]=tree_dic[key]
            mergeditr=mergeditr+1
    final_array=[]
    count=0
    for key in merged_clusters:
        for itr in range(len(merged_clusters[key])):
            array=merged_clusters[key]
            final_array.insert(count,array[itr])
            count=count+1
    '''merged_clustersarray=util.convert_dict_to_list(merged_clusters,len(merged_clusters),len(tree_dic[0]))'''
    centroid = np.mean(final_array, axis=0)


    #anomaly score by subtracting value from centroid
    for anoitr in range(len(final_array)):
        anomalyscore=np.linalg.norm(final_array[anoitr]-centroid)
        anomalyscore_dict[anomalyscore]=final_array[anoitr]

    normal_cluster_cent=get_large_cluster_centroid(small_cluster_threshold,tree_dic)
    for anoitr in range(len(final_array)):
        anomalyscore_largeclusters=np.linalg.norm(final_array[anoitr]-normal_cluster_cent)
        anomalyscore_from_largeclusters[anomalyscore_largeclusters]=final_array[anoitr]
    centroid_array=get_large_cluster_center(small_cluster_threshold,tree_dic)
    # '''get cblof : cluster based local outlier factor.. which calculates the min distance of the data points belong to the smallest clusters to the largest clusters centriod'''
    cblof={}
    for anoitr in range(len(final_array)):
        min=math.inf
        for clusitr in range(len(centroid_array)):

         cblof_dist=np.linalg.norm(final_array[anoitr]-centroid_array[clusitr])
         if cblof_dist<min:
             min=cblof_dist
         if  clusitr==len(centroid_array)-1:
             cblof[min] = final_array[anoitr]

    cblof_largecluster=get_large_clusters_anomalyscore(tree_dic,small_cluster_threshold)
    final_merged_cblof={**cblof,**cblof_largecluster}
    anomaly_scores_list = list(final_merged_cblof.keys())




    sorted_scores = np.sort(anomaly_scores_list)
    cumulative_sum = np.cumsum(sorted_scores)
    percentage_sum = (cumulative_sum / cumulative_sum[-1]) * 100

    knee = KneeLocator(range(1, len(percentage_sum) + 1), percentage_sum, curve="convex", direction="increasing")
    knee_threshold = sorted_scores[knee.elbow]

    y_anom = [data_point for anomaly_score, data_point in final_merged_cblof.items() if anomaly_score > knee_threshold]

    anomaly_score=[]


    tmp=[]

    count=0


    y_anom = np.vstack(y_anom)
    x_indices = np.arange(len(tmp))
    st_ts = ts.tolist()
    st_ts = sorted(st_ts)
    st_ts = np.array(st_ts)
    st_ft = y_anom.tolist()
    st_ft = sorted(st_ft)
    st_ft = np.array(st_ft)
    indices=[]
    count=0
    for i, element in enumerate(st_ts):
      if ((count<=(len(st_ft)-1)) and np.allclose(st_ft[count], element,atol=1e-04)):
          indices.append(i)
          while i + 1 < len(st_ts) and np.allclose(st_ts[i], st_ts[i + 1],atol=1e-04):
              indices.append(i + 1)
              i += 1
          count += 1

    print(count)

    y_pred_test = np.zeros(len(ts), dtype=int)

    y_pred_test[indices]=1
    print(y_pred_test)
    fpr10, tpr10, thresholds = roc_curve(actual, y_pred_test)
    auc_roc_db = auc(fpr10, tpr10)
    recall_db = recall_score(actual, y_pred_test)
    precision_db = precision_score(actual, y_pred_test)
    f1_score_db = f1_score(actual, y_pred_test)

    print()
    print(" Precision:{:.4f}".format(precision_db))
    print(" Recall:{:.4f}".format(recall_db))
    print(" F1-score:{:.4f}".format(f1_score_db))
    print(" AUC-ROC:{:.4f}".format(auc_roc_db))


